In [1]:
import numpy as np, pandas as pd, matplotlib as mtl, zipfile
import matplotlib.pyplot as plt
import os
import re
from sklearn.model_selection import train_test_split

In [2]:
#Class, which unpacks files from zip, rar archives
class Unziper:
    def __init__(self,path):
        self.path=path
        self.check_path=self.__unpacker()
   
    def __unpacker(self):
        def chech_path_existance(self):
            if os.path.exists(self.path):
                def file_extracter(self):
                    list_to_unpack=os.listdir(self.path)
                    if list_to_unpack:
                        other_formats_list=[]
                        for file in list_to_unpack:
                            if re.findall('\.(zip)|(rar)',file):
                                with zipfile.ZipFile(os.path.join(self.path,file)) as zf:
                                    zf.extractall(self.path)
                                result_to_unpack='Upacked'
                            else:
                                other_formats_list.append(file)
                        if len(list_to_unpack)==len(other_formats_list):
                            result_to_unpack='Nothing to unpack'
                    else:
                        result_to_unpack='Nothing to unpack'
                    return result_to_unpack
                return file_extracter(self)
            else:
                result='Path does not exist'
                return result
            
        return chech_path_existance(self)

In [3]:
file_unziper=Unziper(r'C:\Users\pc\Desktop\ML_projects\Digit Recognizer Competition')

In [4]:
file_unziper.check_path

'Upacked'

In [5]:
#Class, which creates dictionaries of files
class FileReader:
    def __init__(self,path):
        self.path=path
        self.read_flat_file=self.create_dict()
        
    def create_dict(self):
        if os.listdir(self.path):
            dict_={}
            for file in os.listdir(self.path):
                if re.findall('\.csv',file):
                    read_file=pd.read_csv(os.path.join(self.path,file))
                    key=re.findall('(\w+)\.',file)[0]
                    dict_[key]=read_file
            if len(dict_):
                return dict_
            else:
                result='Nothing to read'
        else:
            result='Nothing to read'
            return result

In [6]:
file_reader=FileReader(r'C:\Users\pc\Desktop\ML_projects\Digit Recognizer Competition')

In [7]:
files=file_reader.read_flat_file

In [8]:
sample_submission,test,train=files['sample_submission'],files['test'],files['train']

<h5>Datat preparation</h5>

In [9]:
#Class, which shuffles and splits dataset
class ShuffleSplit:
    def __init__(self,random_state,validation_size):
        self.random_state=random_state
        self.validation_size=validation_size
    

    def __split(self,shuffled_dataset):
        upper_border=int(np.ceil(len(shuffled_dataset)*(1-self.validation_size)))
        train_dataset=shuffled_dataset[:upper_border]
        validation_dataset=shuffled_dataset[upper_border:]
        return train_dataset,validation_dataset
            
    def shuffle_split(self,dataset):
        new_order=np.random.RandomState(seed=self.random_state).permutation(dataset.shape[0])
        new_dataset=dataset.values[new_order]
        
            
        return self.__split(new_dataset)

In [10]:
shuffle_split=ShuffleSplit(random_state=42,validation_size=0.2)

In [11]:
train_dataset,validation_dataset=shuffle_split.shuffle_split(train)

In [12]:
train_dataset_no_labels,validation_dataset_no_labels,train_dataset_labels,validation_dataset_labels=train_dataset[:,1:],validation_dataset[:,1:],train_dataset[:,0].reshape(-1,1),validation_dataset[:,0].reshape(-1,1)

In [13]:
train_dataset_no_labels.shape,validation_dataset_no_labels.shape,train_dataset_labels.shape,validation_dataset_labels.shape

((33600, 784), (8400, 784), (33600, 1), (8400, 1))

In [14]:
#Class, which normilizes or standardizes datasets depending on the strategy chosen 
class Scaler:
    def __init__(self,strategy):
        if strategy=='Normalization' or strategy=='Standardization':
            self.__strategy=strategy
        else:
            raise Exception ('There is no such strategy yet')
            
    def scale(self,train_dataset=False,test_dataset=False):
        if np.any(train_dataset) and not np.any(test_dataset):
            calculations=self.__make_calculations_on_train(train_dataset)
            return calculations
        elif np.any(test_dataset) and not np.any(train_dataset):
            calculations=self.__make_calculations_on_test(test_dataset)
            return calculations
        else:
            raise Exception('Train or test datasets should be passes')
        
             
    def __make_calculations_on_train(self,train_dataset):       
        if self.__strategy=='Normalization':
            self.train_dataset_min=train_dataset.min(axis=0)
            self.train_dataset_max=train_dataset.max(axis=0)
            result=(train_dataset-self.train_dataset_min)/(self.train_dataset_max-self.train_dataset_min)
            return np.nan_to_num(result)
        elif self.__strategy=='Standardization':
            self.train_dataset_mean=train_dataset.mean(axis=0)
            self.train_dataset_std=train_dataset.std(axis=0)
            result=(train_dataset-self.train_dataset_mean)/self.train_dataset_std
            return np.nan_to_num(result)
            
    
    def __make_calculations_on_test(self,test_dataset):
        if self.__strategy=='Normalization':
            result=(test_dataset-self.train_dataset_min)/(self.train_dataset_max-self.train_dataset_min)
            return np.nan_to_num(result)
        elif self.__strategy=='Standardization':
            result=(test_dataset-self.train_dataset_mean)/self.train_dataset_std
            return np.nan_to_num(result)

In [15]:
scaler_norm=Scaler(strategy='Normalization')
scaler_stand=Scaler(strategy='Standardization')

In [16]:
train_dataset_no_labels_normolized=scaler_norm.scale(train_dataset=train_dataset_no_labels)
train_dataset_no_labels_standardized=scaler_stand.scale(train_dataset=train_dataset_no_labels)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


In [17]:
validation_dataset_no_labels_normolized=scaler_norm.scale(test_dataset=validation_dataset_no_labels)
validation_dataset_no_labels_standardized=scaler_norm.scale(test_dataset=validation_dataset_no_labels)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


<h5>Making predictions</h5>

In [20]:
#The class below runs cross_validation, fits models, predicts target labels
class FitPredictorEstimator:
    def __init__(self,prediction_strategy,estimation_strategy,specify_estimation_strategy=False,specify_random_state=False,specify_cv=False,specify_num_estimators=False,specify_max_depth=False):
        self.__prediction_strategy=prediction_strategy
        self.__estimation_strategy=estimation_strategy
        self.__specify_estimation_strategy=specify_estimation_strategy
        self.__specify_random_state=specify_random_state
        self.__specify_cv=specify_cv
        self.__specify_num_estimators=specify_num_estimators
        self.__specify_max_depth=specify_max_depth
        
        
        if self.__prediction_strategy=='sgd_classifier':
            from sklearn.linear_model import SGDClassifier
            if not self.__specify_random_state:
                self.__model=SGDClassifier()
            else:
                self.__model=SGDClassifier(random_state=self.__specify_random_state)
        elif self.__prediction_strategy=='random_forest_classifier':
            from sklearn.ensemble import RandomForestClassifier
            if not self.__specify_num_estimators and not self.__specify_max_depth:
                self.__model=RandomForestClassifier()
            elif self.__specify_num_estimators and self.__specify_max_depth:
                self.__model=RandomForestClassifier(n_estimators=self.__specify_num_estimators,max_depth=self.__specify_max_depth)
            elif self.__specify_num_estimators and not self.__specify_max_depth:
                self.__model=RandomForestClassifier(n_estimators=self.__specify_num_estimators)
            elif not self.__specify_num_estimators and self.__specify_max_depth:
                elf.__model=RandomForestClassifier(max_depth=self.__specify_max_depth)
        else:
            raise Exception ('There is no such strategy yet')
            
    def cross_validate_estimate(self,train_dataset,train_labels):
        if self.__specify_cv:
            from sklearn.model_selection import cross_val_predict
            self.predicted_train_labels=cross_val_predict(self.__model,train_dataset,train_labels,cv=self.__specify_cv)
            self.cross_validate_estimation=self.__estimate(train_labels,self.predicted_train_labels)
        else:
            raise Exception ('Method works with k-folds cross validation, please, use specify_cv, while creating an object of Class')
            
            
    def __estimate(self,labels,predicted_labels):
        if self.__estimation_strategy=='f1_score':
            from sklearn.metrics import f1_score
            return f1_score(labels,predicted_labels,average=self.__specify_estimation_strategy)
        else:
            raise Except ('There is no such strategy yet')
            
            
    def fit(self,train_dataset,train_labels):
        self.__model.fit(train_dataset,train_labels)
        
    def predict(self,dataset):
        self.__predictions=self.__model.predict(dataset)
        return self.__predictions
        
    def estimate(self,validation_labels):
        self.validation_estimation=self.__estimate(validation_labels,self.__predictions)
        return self.validation_estimation

In [21]:
predictor_estimator_normalized_linear=FitPredictorEstimator(prediction_strategy='sgd_classifier',
                                         estimation_strategy='f1_score',
                                         specify_random_state=42, specify_estimation_strategy='weighted',specify_cv=5
                                         )

In [22]:
predictor_estimator_normalized_linear.cross_validate_estimate(train_dataset_no_labels_normolized,train_dataset_labels.ravel())

In [23]:
#Let's calculate estimations on normolized training set
predictor_estimator_normalized_linear.cross_validate_estimation

0.904743495917294

In [24]:
#Well, not too bad, let's now train our model and make predictions on validation set
predictor_estimator_normalized_linear.fit(train_dataset_no_labels_normolized,train_dataset_labels.ravel())

In [25]:
predictor_estimator_normalized_linear.predict(validation_dataset_no_labels_normolized)

C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


array([5, 2, 4, ..., 2, 6, 0], dtype=int64)

In [26]:
#Our model can generalize, since the estimation on validation set is approximately the same as on the training set and even higher 
predictor_estimator_normalized_linear.estimate(validation_dataset_labels)

0.9098029276790895

In [28]:
#Now let's make the same steps using other model

In [34]:
predictor_estimator_normalized_forest=FitPredictorEstimator(prediction_strategy='random_forest_classifier',
                                         estimation_strategy='f1_score',
                                         specify_estimation_strategy='weighted',specify_cv=5,specify_num_estimators=50
                                         )

In [35]:
predictor_estimator_normalized_forest.cross_validate_estimate(train_dataset_no_labels_normolized,train_dataset_labels.ravel())

In [36]:
#Random Forest Classifier performs better, than linear model
predictor_estimator_normalized_forest.cross_validate_estimation

0.9591612302746899

In [37]:
predictor_estimator_forest=FitPredictorEstimator(prediction_strategy='random_forest_classifier',
                                         estimation_strategy='f1_score',
                                         specify_estimation_strategy='weighted',specify_cv=5,specify_num_estimators=50
                                         )

In [38]:
predictor_estimator_forest.cross_validate_estimate(train_dataset_no_labels,train_dataset_labels.ravel())

In [39]:
#Let us experiment a bit with random forest classifier: the class object, which uses normolized dataset was launched above, and
#here not normolized dataset was used. As it comes to decision trees or forest the scale of the data should make no difference
#to the result of the model, and we can see that it makes almost no. Almost, that means, that there is a slight difference between
#these two models
predictor_estimator_forest.cross_validate_estimation

0.9583754696757832

In [ ]:
#Since random forest performs better, than sgd, and there is almost no difference between models on scaled and not scaled data, 
#as it comes to forest, let's use random forest with not scaled training data in further steps

In [41]:
predictor_estimator_forest.fit(train_dataset_no_labels,train_dataset_labels.ravel())

In [42]:
predictor_estimator_forest.predict(validation_dataset_no_labels)

array([5, 2, 4, ..., 2, 6, 0], dtype=int64)

In [43]:
predictor_estimator_forest.estimate(validation_dataset_labels)

0.9629459774325437

In [45]:
test_predictions=predictor_estimator_forest.predict(test)

In [59]:
test_predictions_dataset=pd.DataFrame(test_predictions,columns=['Label'])

In [75]:
test_predictions_dataset['ImageId']=[i for i in range(1,test_predictions_dataset.shape[0]+1)]

In [76]:
test_predictions_dataset=test_predictions_dataset[['ImageId','Label']]

In [77]:
test_predictions_dataset

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [78]:
test_predictions_dataset.to_csv(r'C:\Users\pc\Desktop\ML_projects\Digit Recognizer Competition_result\result_v2.csv',index=False)